In [2]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier


In [3]:
name = pd.read_csv("name_500.csv")
name

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0050419,tt0072308,tt0053137"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0037382,tt0117057,tt0038355,tt0075213"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0049189,tt0054452,tt0056404,tt0057345"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0078723,tt0080455,tt0072562,tt0077975"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050976,tt0050986,tt0069467,tt0083922"
...,...,...,...,...,...,...
495,nm0000496,Juliette Lewis,1973,\N,"actress,soundtrack,director","tt1322269,tt0116367,tt0101540,tt0110632"
496,nm0000497,Jennifer Lien,1974,\N,"actress,producer,script_department","tt0112178,tt0133189,tt0106100,tt0120586"
497,nm0000498,Matthew Lillard,1970,\N,"actor,producer,soundtrack","tt0331632,tt0267913,tt0133189,tt1033575"
498,nm0000499,Bai Ling,\N,\N,"actress,producer,writer","tt0109506,tt0119994,tt0405336,tt1121931"


In [4]:
title_akas = pd.read_csv("title_akas_500.csv")
title_akas

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0
...,...,...,...,...,...,...,...,...
495,tt0000077,1,"Le fakir, mystère indien",\N,\N,original,\N,1
496,tt0000077,2,"Le fakir, mystère indien",FR,\N,imdbDisplay,\N,0
497,tt0000078,1,Feira de Gado na Corujeira,PT,\N,imdbDisplay,\N,0
498,tt0000078,2,Feira de Gado na Corujeira,\N,\N,original,\N,1


In [5]:
title_principle = pd.read_csv("title.principals_500.csv")
title_principle

,tconst,ordering,nconst,category,job,characters
0,tt3583650,9,nm3938687,self,\N,"[""Self - Host""]"
1,tt4919780,10,nm1729049,editor,\N,\N
2,tt4729596,7,nm12240212,self,\N,"[""Self""]"
3,tt16270064,3,nm5689837,director,\N,\N
4,tt6623958,3,nm3651422,director,\N,\N
...,...,...,...,...,...,...
495,tt14440488,7,nm12330168,writer,dialogue,\N
496,tt3243324,8,nm5084460,cinematographer,\N,\N
497,tt13325264,1,nm9821305,actor,\N,"[""Faldo""]"
498,tt11353632,7,nm1224445,composer,\N,\N


In [6]:
name = pd.DataFrame({'nconst': ['nm1', 'nm2', 'nm3'],
                     'primaryName': ['John Doe', 'Jane Doe', 'Alice'],
                     'birthYear': [1980, 1990, 1975]})
title = pd.DataFrame({'tconst': ['tt1', 'tt2', 'tt3'],
                      'ordering_title': [1, 2, 3],  # Change the name of the ordering column in 'title'
                      'title': ['Movie 1', 'Movie 2', 'Movie 3']})
crew = pd.DataFrame({'tconst': ['tt1', 'tt2', 'tt3'],
                     'ordering_crew': [1, 2, 3],  # Change the name of the ordering column in 'crew'
                     'nconst': ['nm1', 'nm2', 'nm3'],
                     'category': ['actor', 'actress', 'director']})

# Assuming your target variable is in the 'category' column of the crew DataFrame
target_column = 'category'

# Feature engineering steps
name_features = ['birthYear']
title_features = ['ordering_title']  # Adjust the column name
crew_features = ['ordering_crew']  # Adjust the column name

# Create feature union transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('name', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), name_features),
        ('title', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
            ('scaler', StandardScaler())
        ]), title_features),
        ('crew', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
            ('scaler', StandardScaler())
        ]), crew_features),
    ])

# Create the pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())
])

# Assuming X is your feature matrix and y is your target variable
X = pd.concat([name[name_features], title[title_features], crew[crew_features]], axis=1)
y = crew[target_column]

# Fit the pipeline
pipeline.fit(X, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('name',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['birthYear']),
                                                 ('title',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy='constant')),
                                                                  ('scaler',
                                                                   StandardScaler())]